In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
%cd "../.."

/home/hazzu/Code/opencv_streamlit


# 1. Define function


In [4]:
import os
import cv2
import numpy as np
from typing import Tuple, List
from services.semantic_keypoint_detection.services import DATATYPES


def read_image(type: int, name: str):
    """
    Reads an image and its ground truth keypoints from the specified dataset.

    :param type: The type of the dataset.
    :param name: The name of the image.

    :return (image, ground_truth): A tuple containing the image and the ground truth keypoints.
    """
    image = cv2.imread(os.path.join(DATATYPES[type], "images", f"{name}.png"))
    ground_truth = np.load(os.path.join(DATATYPES[type], "points", f"{name}.npy"))
    ground_truth = [cv2.KeyPoint(y, x, 1, 0, 0, 0) for x, y in ground_truth]
    return (image, ground_truth)


def rotate_image(image: cv2.typing.MatLike, angle: int) -> cv2.typing.MatLike:
    """
    Rotates an image by a given angle.

    :param image: The image to rotate.
    :param angle: The angle to rotate the image by.

    :return rotated_image: The rotated image.
    """
    h, w = image.shape[:2]
    matrix_rotation = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, matrix_rotation, (w, h))
    return rotated_image


def rotate_keypoints(
    size: Tuple[int, int], keypoints: List[cv2.KeyPoint], angle: int
) -> List[cv2.KeyPoint]:
    """
    Rotates the keypoints of an image.

    :param size: The size of the image (witdh, height).
    :param keypoints: The keypoints to rotate.
    :param angle: The angle to rotate the keypoints by.

    :return (result, idx): A tuple containing the rotated keypoints and their indices in the original list.
    """
    matrix_rotation = cv2.getRotationMatrix2D((size[0] / 2, size[1] / 2), angle, 1)
    kps = np.array([[kp.pt[0], kp.pt[1]] for kp in keypoints])
    kps = np.concatenate([kps, np.ones((len(kps), 1))], axis=1)
    rotated_kps = np.array(np.dot(matrix_rotation, kps.T)).T

    result, idx = [], []
    for i in range(len(rotated_kps)):
        kp = rotated_kps[i]
        if 0 <= kp[0] < size[0] and 0 <= kp[1] < size[1]:
            result.append(cv2.KeyPoint(kp[0], kp[1], 1, 0, 0, 0))
            idx.append(i)

    return (result, idx)

# 2. Cache descriptors of original image


In [18]:
import cv2
from tqdm import tqdm
from services.keypoint_matching.superpoint import SuperPointFrontend


def cache_descriptors(model, convert=False):
    """
    Caches the descriptors of the images in the datasets.

    :param model: The model to use for the caching.

    :return cache_descriptors: The cached descriptors.
    """
    cache_descriptors = []

    for i in range(8):
        cache_descriptors.append([])

        for j in tqdm(range(500), desc=f"Cache descriptors for shape type {i}"):
            image, ground_truth = read_image(i, str(j))
            gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            if convert:
                gray_scale = gray_scale.astype(np.float32) / 255.0

            if len(ground_truth) > 0:
                descriptors = model.compute(gray_scale, ground_truth)[1]
            else:
                descriptors = None

            if descriptors is not None:
                assert descriptors.shape[0] == len(
                    ground_truth
                ), f"{descriptors.shape[0]} != {len(ground_truth)}"
            else:
                assert len(ground_truth) == 0

            cache_descriptors[i].append(descriptors)

    return cache_descriptors


print("Caching descriptors for SIFT...")
sift = cv2.SIFT_create()
cache_sift_descriptors = cache_descriptors(sift)

print("Caching descriptors for ORB...")
orb = cv2.ORB_create(edgeThreshold=0, fastThreshold=0)
cache_orb_descriptors = cache_descriptors(orb)

print("Caching descriptors for SuperPoint...")
fe = SuperPointFrontend(
    weights_path="./services/keypoint_matching/superpoint_v1.pth",
    nms_dist=4,
    conf_thresh=0.015,
    nn_thresh=0.7,
    cuda=True,
)
cache_superpoint_descriptors = cache_descriptors(fe, True)

Caching descriptors for SIFT...


Cache descriptors for shape type 7: 100%|██████████| 500/500 [00:00<00:00, 1735.02it/s]


Caching descriptors for ORB...


Cache descriptors for shape type 7: 100%|██████████| 500/500 [00:00<00:00, 3414.82it/s]
/home/hazzu/Code/opencv_streamlit/services/keypoint_matching/superpoint.py:79: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues 

Caching descriptors for SuperPoint...
Run on GPU


Cache descriptors for shape type 7: 100%|██████████| 500/500 [00:01<00:00, 411.19it/s]


# 3. Get accuracy matching keypoints of SIFT


In [6]:
sift = cv2.SIFT_create()
bf_sift = cv2.BFMatcher(cv2.NORM_L2)
accuracy_matches_sift = []

for angle in range(0, 360, 10):
    accuracy_matches_sift.append([])

    for type in range(8):
        accuracy_matches_sift[-1].append([])

        for i in tqdm(range(500), f"Angle: {angle} - Shape type: {type}"):
            image, ground_truth = read_image(type, str(i))
            gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            descriptor = cache_sift_descriptors[type][i]

            if descriptor is None:
                continue

            # Rotate the image and the keypoints.
            rotated_image = rotate_image(gray_scale, angle)
            rotated_keypoints, idx = rotate_keypoints(
                (image.shape[1], image.shape[0]), ground_truth, angle
            )

            # Compute the descriptors of the rotated image.
            rotated_descriptor = sift.compute(rotated_image, rotated_keypoints)[1]
            if rotated_descriptor is not None:
                assert len(rotated_descriptor) == len(rotated_keypoints)
            else:
                assert len(rotated_keypoints) == 0
                continue

            # Get the original keypoints and descriptors.
            original_descriptor = np.array(descriptor)[idx]

            # Match the descriptors of the original and rotated images.
            if len(rotated_descriptor) == 1:
                accuracy_matches_sift[-1][-1].append(1)
            else:
                matches = bf_sift.knnMatch(rotated_descriptor, original_descriptor, k=2)
                matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
                accuracy_matches_sift[-1][-1].append(
                    len(matches) / len(original_descriptor)
                )

        if len(accuracy_matches_sift[-1][-1]) > 0:
            print(f"Accuracy: {np.mean(accuracy_matches_sift[-1][-1])}")
        else:
            print("No matches found.")

Angle: 0 - Shape type: 0: 100%|██████████| 500/500 [00:00<00:00, 929.72it/s]


Accuracy: 1.0


Angle: 0 - Shape type: 1: 100%|██████████| 500/500 [00:00<00:00, 915.69it/s] 


Accuracy: 1.0


Angle: 0 - Shape type: 2: 100%|██████████| 500/500 [00:00<00:00, 2861.87it/s]


...

Accuracy: 0.9950333333333333


Angle: 350 - Shape type: 6: 100%|██████████| 500/500 [00:00<00:00, 959.16it/s] 


Accuracy: 0.9640666666666666


Angle: 350 - Shape type: 7: 100%|██████████| 500/500 [00:00<00:00, 1265.33it/s]

Accuracy: 0.764746238609875


In [7]:
for i in range(36):
    print(f"Angle: {i * 10}")
    flatten = []
    for j in range(8):
        flatten += accuracy_matches_sift[i][j]
    print(f"Accuracy: {np.mean(flatten)}")

Angle: 0
Accuracy: 0.99940581098872
Angle: 10
Accuracy: 0.8235145681615308
Angle: 20
Accuracy: 0.6226938884247238
Angle: 30
Accuracy: 0.43811179674688433
Angle: 40
Accuracy: 0.3146604669856603
Angle: 50
Accuracy: 0.28553740148898776
Angle: 60
Accuracy: 0.29886299781863196
Angle: 70
Accuracy: 0.3199396870015503
Angle: 80
Accuracy: 0.34469590949583956
Angle: 90
Accuracy: 0.36709617334934647
Angle: 100
Accuracy: 0.38117664699102927
Angle: 110
Accuracy: 0.3834172551508624
Angle: 120
Accuracy: 0.3826890701767783
Angle: 130
Accuracy: 0.3680111715598673
Angle: 140
Accuracy: 0.3440374364559226
Angle: 150
Accuracy: 0.33281942118418173
Angle: 160
Accuracy: 0.3355497279444249
Angle: 170
Accuracy: 0.3597741052325764
Angle: 180
Accuracy: 0.37339379428086716
Angle: 190
Accuracy: 0.3660577596904552
Angle: 200
Accuracy: 0.33893121586927927
Angle: 210
Accuracy: 0.3321925956290665
Angle: 220
Accuracy: 0.34627161777019083
Angle: 230
Accuracy: 0.36795575109652157
Angle: 240
Accuracy: 0.38120525250936366
A

# 4. Get accuracy matching keypoints of ORB


In [8]:
orb = cv2.ORB_create(edgeThreshold=0, fastThreshold=0)
bf_orb = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
accuracy_matches_orb = []

for angle in range(0, 360, 10):
    accuracy_matches_orb.append([])

    for type in range(8):
        accuracy_matches_orb[-1].append([])

        for i in tqdm(range(500), f"Angle: {angle} - Shape type: {type}"):
            image, ground_truth = read_image(type, str(i))
            gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            descriptor = cache_orb_descriptors[type][i]

            if descriptor is None:
                continue

            # Rotate the image and the keypoints.
            rotated_image = rotate_image(gray_scale, angle)
            rotated_keypoints, idx = rotate_keypoints(
                (image.shape[1], image.shape[0]), ground_truth, angle
            )

            # Compute the descriptors of the rotated image.
            rotated_descriptor = orb.compute(rotated_image, rotated_keypoints)[1]
            if rotated_descriptor is not None:
                assert len(rotated_descriptor) == len(rotated_keypoints)
            else:
                assert len(rotated_keypoints) == 0
                continue

            # Get the original keypoints and descriptors.
            original_descriptor = np.array(descriptor)[idx]

            # Match the descriptors of the original and rotated images.
            if len(rotated_descriptor) == 1:
                accuracy_matches_orb[-1][-1].append(1)
            else:
                matches = bf_orb.match(rotated_descriptor, original_descriptor)
                accuracy_matches_orb[-1][-1].append(
                    len(matches) / len(original_descriptor)
                )

        if len(accuracy_matches_orb[-1][-1]) > 0:
            print(f"Accuracy: {np.mean(accuracy_matches_orb[-1][-1])}")
        else:
            print("No matches found.")

Angle: 0 - Shape type: 0: 100%|██████████| 500/500 [00:00<00:00, 1252.41it/s]


Accuracy: 0.9994561736690857


Angle: 0 - Shape type: 1: 100%|██████████| 500/500 [00:00<00:00, 1521.46it/s]


Accuracy: 1.0


Angle: 0 - Shape type: 2: 100%|██████████| 500/500 [00:00<00:00, 2926.50it/s]


...

Accuracy: 0.9917


Angle: 350 - Shape type: 6: 100%|██████████| 500/500 [00:00<00:00, 1374.00it/s]


Accuracy: 0.9956333333333335


Angle: 350 - Shape type: 7: 100%|██████████| 500/500 [00:00<00:00, 1894.52it/s]

Accuracy: 0.8669428214882761


In [9]:
for i in range(36):
    print(f"Angle: {i * 10}")
    flatten = []
    for j in range(8):
        flatten += accuracy_matches_orb[i][j]
    print(f"Accuracy: {np.mean(flatten)}")

Angle: 0
Accuracy: 0.9996223863455721
Angle: 10
Accuracy: 0.9066339243806969
Angle: 20
Accuracy: 0.7535621182542792
Angle: 30
Accuracy: 0.5558078223668997
Angle: 40
Accuracy: 0.4946471715763078
Angle: 50
Accuracy: 0.49017296835018703
Angle: 60
Accuracy: 0.49441089332912175
Angle: 70
Accuracy: 0.5024697508433382
Angle: 80
Accuracy: 0.5072816182678902
Angle: 90
Accuracy: 0.5109708512520462
Angle: 100
Accuracy: 0.5129041481963077
Angle: 110
Accuracy: 0.5111375560070245
Angle: 120
Accuracy: 0.5119789906183833
Angle: 130
Accuracy: 0.5025277607586435
Angle: 140
Accuracy: 0.49506827113710977
Angle: 150
Accuracy: 0.4930611921233538
Angle: 160
Accuracy: 0.4955456607388959
Angle: 170
Accuracy: 0.5147581345851822
Angle: 180
Accuracy: 0.5353868331070607
Angle: 190
Accuracy: 0.5154041101435454
Angle: 200
Accuracy: 0.4984902498801409
Angle: 210
Accuracy: 0.49574669393488136
Angle: 220
Accuracy: 0.49894646125395625
Angle: 230
Accuracy: 0.5048346258291043
Angle: 240
Accuracy: 0.5134624227419778
Angle:

# 5. Get accuracy matching keypoints of SuperPoint


In [19]:
from services.keypoint_matching.superpoint import SuperPointFrontend

fe = SuperPointFrontend(
    weights_path="./services/keypoint_matching/superpoint_v1.pth",
    nms_dist=4,
    conf_thresh=0.015,
    nn_thresh=0.7,
    cuda=True,
)
bf_sift = cv2.BFMatcher(cv2.NORM_L2)
accuracy_matches_superpoint = []

for angle in range(0, 360, 10):
    accuracy_matches_superpoint.append([])

    for type in range(8):
        accuracy_matches_superpoint[-1].append([])

        for i in tqdm(range(500), f"Angle: {angle} - Shape type: {type}"):
            image, ground_truth = read_image(type, str(i))
            gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray_scale = gray_scale.astype(np.float32) / 255.0
            descriptor = cache_superpoint_descriptors[type][i]

            if descriptor is None:
                continue

            # Rotate the image and the keypoints.
            rotated_image = rotate_image(gray_scale, angle)
            rotated_keypoints, idx = rotate_keypoints(
                (image.shape[1], image.shape[0]), ground_truth, angle
            )

            # Compute the descriptors of the rotated image.
            rotated_descriptor = fe.compute(rotated_image, rotated_keypoints)[1]
            if rotated_descriptor is not None:
                assert len(rotated_descriptor) == len(rotated_keypoints)
            else:
                assert len(rotated_keypoints) == 0
                continue

            # Get the original keypoints and descriptors.
            original_descriptor = np.array(descriptor)[idx]

            # Match the descriptors of the original and rotated images.
            if len(rotated_descriptor) == 1:
                accuracy_matches_superpoint[-1][-1].append(1)
            else:
                matches = bf_sift.knnMatch(rotated_descriptor, original_descriptor, k=2)
                matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
                accuracy_matches_superpoint[-1][-1].append(
                    len(matches) / len(original_descriptor)
                )

        if len(accuracy_matches_superpoint[-1][-1]) > 0:
            print(f"Accuracy: {np.mean(accuracy_matches_superpoint[-1][-1])}")
        else:
            print("No matches found.")

/home/hazzu/Code/opencv_streamlit/services/keypoint_matching/superpoint.py:79: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(weights_path

Run on GPU


Angle: 0 - Shape type: 0:   0%|          | 0/500 [00:00<?, ?it/s]/home/hazzu/anaconda3/envs/opencv_streamlit/lib/python3.12/site-packages/torch/nn/functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
Angle: 0 - Shape type: 0: 100%|██████████| 500/500 [00:02<00:00, 248.96it/s]


Accuracy: 1.0


Angle: 0 - Shape type: 1: 100%|██████████| 500/500 [00:01<00:00, 276.75it/s]


...

Accuracy: 0.9841666666666667


Angle: 350 - Shape type: 6: 100%|██████████| 500/500 [00:01<00:00, 291.88it/s]


Accuracy: 0.9933


Angle: 350 - Shape type: 7: 100%|██████████| 500/500 [00:01<00:00, 446.55it/s]

Accuracy: 0.8316574755211119


In [20]:
for i in range(36):
    print(f"Angle: {i * 10}")
    flatten = []
    for j in range(8):
        flatten += accuracy_matches_superpoint[i][j]
    print(f"Accuracy: {np.mean(flatten)}")

Angle: 0
Accuracy: 1.0
Angle: 10
Accuracy: 0.9381710854929227
Angle: 20
Accuracy: 0.860586843094134
Angle: 30
Accuracy: 0.6125228412631787
Angle: 40
Accuracy: 0.30216296504376255
Angle: 50
Accuracy: 0.15222644640539418
Angle: 60
Accuracy: 0.1529840832611402
Angle: 70
Accuracy: 0.18952787302448829
Angle: 80
Accuracy: 0.21669352478558548
Angle: 90
Accuracy: 0.23723531440093487
Angle: 100
Accuracy: 0.24997934283450984
Angle: 110
Accuracy: 0.2508251794470871
Angle: 120
Accuracy: 0.2494037773060901
Angle: 130
Accuracy: 0.24317749145956744
Angle: 140
Accuracy: 0.22427176489388356
Angle: 150
Accuracy: 0.21478265445147277
Angle: 160
Accuracy: 0.22736999955835926
Angle: 170
Accuracy: 0.2399486580515641
Angle: 180
Accuracy: 0.249575608841945
Angle: 190
Accuracy: 0.2413977108560041
Angle: 200
Accuracy: 0.22835949620821414
Angle: 210
Accuracy: 0.2165405917944215
Angle: 220
Accuracy: 0.22200101114323462
Angle: 230
Accuracy: 0.24308610008382125
Angle: 240
Accuracy: 0.2466354149180754
Angle: 250
Accu

# 6. Convert result to Numpy array


In [21]:
# results_sift = []
# results_orb = []
results_superpoint = []

for i in range(36):
    # flatten = []
    # for j in range(8):
    #     flatten += accuracy_matches_sift[i][j]
    # results_sift.append(np.mean(flatten))

    # flatten = []
    # for j in range(8):
    #     flatten += accuracy_matches_orb[i][j]
    # results_orb.append(np.mean(flatten))

    flatten = []
    for j in range(8):
        flatten += accuracy_matches_superpoint[i][j]
    results_superpoint.append(np.mean(flatten))

# np.save("./services/keypoint_matching/results_sift.npy", results_sift)
# np.save("./services/keypoint_matching/results_orb.npy", results_orb)
np.save("./services/keypoint_matching/results_superpoint.npy", results_superpoint)